# Dataset Preparation

In the [supervised_learning notebook](supervised_learning.ipynb), we compare and analyze the results for different types of models applied to a supervised learning task. The present notebook outlines the basic steps to prepare the dataset for the execution of this learning task.

For simplicity and familiarity, we will use the dataset **UK Traffic Accidents** that was used in the assignment 1. For more details: [Assignmet 1 Github Page](https://github.com/agomez08/patrones_inv1).

As a goal, we plan on using this accidents data to attempt to **predict the severity of the accident given the relevant features in the dataset**. We will start with an exported CSV file from the pre-processing applied in the previous assignment called *'uk_accidents_processed.csv'*. We will take this pre-processing further now that there is a specific application in mind.

## Loading the Data

In [ ]:
# Start by importing relevant python modules
import pandas as pd
import sklearn

In [ ]:
# Read exported data from previous investigation
df_accidents_05_14 = pd.read_csv('dataset/uk_accidents_processed.csv')

Below we identify the number of instances for each of the possible severities. From the dataset documentation and the changes applied on the pre-processing already applied, this is what each of the numbers mean:

1 - Slight severity.

2 - Serious severity.

3 - Fatal severity.

In [ ]:
# Count by each type
df_accidents_05_14['Accident_Severity'].value_counts()

## Categories Reduction

To simplify the prediction task and to start balancing the classes, we will reduce the scope of this experiment to attempt to identify an accident as Severe or Non-Severe:

0 - Non-Severe (Slight severity).

1 - Severe (Serious or Fatal severity).

We apply these changes below:

In [ ]:
# Save 1 as 0
df_accidents_05_14.loc[df_accidents_05_14['Accident_Severity'] == 1, 'Accident_Severity'] = 0

# Save 2 and 3 as 1
df_accidents_05_14.loc[df_accidents_05_14['Accident_Severity'] == 2, 'Accident_Severity'] = 1
df_accidents_05_14.loc[df_accidents_05_14['Accident_Severity'] == 3, 'Accident_Severity'] = 1

In [ ]:
# Rename the column to be more representative:
df_accidents_05_14 = df_accidents_05_14.rename(columns={'Accident_Severity': 'Severe_Accident'})

With this change we have the classes distribution shown below:

In [ ]:
# Count by each type
df_accidents_05_14['Severe_Accident'].value_counts()

## Balancing Classes

To balance the classes, and since we have a very high-number of instances anyways, we will down-sample the *Non-Severe* class to have the same number of samples as the *Severe* class. For this purpose we use the *resample* function of *sklearn* which will apply a random method for this purpose.

In [ ]:
# Downsample non-severe category
df_accidents_non_severe = df_accidents_05_14[df_accidents_05_14['Severe_Accident'] == 0] 
df_accidents_non_severe = sklearn.utils.resample(df_accidents_non_severe, replace=False,
                                                 n_samples=71178, random_state=18)
 
# Combine again
df_accidents_severe = df_accidents_05_14[df_accidents_05_14['Severe_Accident'] == 1]
df_accidents = pd.concat([df_accidents_non_severe, df_accidents_severe])

As we can see below, the two classes are now balanced:

In [ ]:
# Count by each type
df_accidents['Severe_Accident'].value_counts()

In [ ]:
# Re-organize and then reset index
df_accidents.sort_index(inplace=True)
df_accidents.reset_index(drop=True, inplace=True)

## Dropping Irrelevant Features

Below we proceeed to remove different columns that due to their nature are not considered relevant for the application we have in mind for this data. Some of them are also consequences of the accidents which are highly related to the severity and wouldn't be available for the prediction (*'Number_of_Vehicles'* and *'Number_of_Casualties'*).

In [ ]:
df_accidents.drop(inplace=True, columns=['Police_Force', 'Number_of_Vehicles', 'Number_of_Casualties', '1st_Road_Class',
                                        '1st_Road_Number', '2nd_Road_Class', '2nd_Road_Number', 'Police_Attended',
                                         'Year', 'Local_Authority_Highway'])

## Storing Resultant Dataset
Below we present a glance of the obtained dataset that will be used to train the supervised learning models. As we can observe it has 142356 instances and 58 features.

In [ ]:
df_accidents

In [ ]:
# Save to separate file called uk_accidents_for_sev_prediction.csv
df_accidents.to_csv('dataset/uk_accidents_for_sev_prediction.csv', index=False)